# Plate Capture

Description
Python version 3.6.8
TensorFlow version 1.12.0
OpenCv 4.0.0
Model Used: SSD MobileNet ssd_mobilenet_v1_coco_2017_11_17

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys

import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
import matplotlib

from matplotlib import pyplot as plt
from PIL import Image
matplotlib.use("TkAgg")
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')
import logging
import sys


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Model exported using the `export_inference_graph.py` tool `FROZEN_GRAPH` loaded to point .pb file.  


In [4]:
#Model used
MODEL_NAME = 'numberPlate'

# Path to frozen detection graph. 
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## loading the image

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
#Test images path
PATH_TO_TEST_IMAGES_DIR = 'images/test'

#PATH_TO_TEST_IMAGES_DIR = '/home/shivam/Desktop/Test/'
#Image for which Numbers from Plate to be detected
#img_get = "1.jpg"


#number are the indexing of the images
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in range(1,2) ] 
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, img_get)]
# Size, in inches, of the output images.
IMAGE_SIZE = (14, 8)

In [9]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      print(output_dict['detection_boxes'][0])
      print(output_dict['detection_scores'][0])
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

# Crop Image 

In [11]:
import cv2

#i = 1
for image_path in TEST_IMAGE_PATHS:
   
    
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    
    #Code to crop the bounding boxed image i.e. Number Plate
    ymin,xmin,ymax,xmax = output_dict['detection_boxes'][0]
    
    (im_width, im_height) = image.size
    (xminn, xmaxx, yminn, ymaxx) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
    cropped_image = tf.image.crop_to_bounding_box(image, int(yminn), int(xminn), int(ymaxx - yminn), int(xmaxx - xminn))
    
    #tensorflow session required to read the tensors
    sess = tf.Session()
    img_data = sess.run(cropped_image)
    sess.close()
    plt.imshow(img_data)
    path,file = os.path.split(image_path)
    
    
    #path to store the croppedimage
    plt.imsave("/home/shivam/Desktop/Test/{}".format(file),img_data)
    
    #i += 1
    
#plt.show()

[0.5242167  0.35398465 0.5951311  0.6222572 ]
0.98220164


In [ ]:
'''import cv2
def into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata())


img = "/home/shivam/Desktop/Test/1.jpg"
image = cv2.imread(img)

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imsave("/home/shivam/Desktop/Test/{}".format("abc1"),gray_image)
ret = Image.open("/home/shivam/Desktop/Test/abc1.png")'''

# To Generate Plate Number

In [ ]:
import sys

sys.path.append("/home/shivam/Downloads/NumberPlateRecognition/")

from Main import *
from DetectChars import *
from DetectPlates import *
from PossibleChar import *
from PossiblePlate import *
from Preprocess import *
#from Classifications import *
#from Flattened_images import *


num()



#Image_Path = "/home/" 
#num(TEST_IMAGE_PATHS) #Function in Main.py stored at NumberPLateRecognition.

In [ ]:
'''import cv2
import matplotlib.pyplot as plt
image = cv2.imread("/home/shivam/Desktop/Test/abc1.png")
imgGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Otsu's thresholding
ret3, th3 = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Gaussian Blur
blur = cv2.GaussianBlur(th3, (5, 5), 0)

i = 99
plt.imsave("/home/shivam/Desktop/Test/{}.jpg".format(i),imgThresh)'''


In [ ]:
'''import sys
import pyvips
import cv2
import matplotlib.pyplot as plt
from PIL import Image
img = cv2.imread("/home/shivam/Desktop/Test/2.jpg")
image = pyvips.Image.new_from_file("/home/shivam/Desktop/Test/2.jpg")

# to monochrome, take the fft, wrap the origin to the centre, get magnitude
fft = image.colourspace('b-w').fwfft().wrap().abs()


def to_rectangular(image):
    xy = pyvips.Image.xyz(image.width, image.height)
    xy *= [1, 360.0 / image.height]
    index = xy.rect()
    scale = min(image.width, image.height) / float(image.width)
    index *= scale / 2.0
    index += [image.width / 2.0, image.height / 2.0]
    return image.mapim(index)

# sum of columns, sum of rows
cols, rows = to_rectangular(fft).project()
v, x, y = rows.gaussblur(10).maxpos()

# and turn to an angle in degrees we should counter-rotate by
angle = 270 - 360 * y / rows.height

img = image.rotate(angle)'''